# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770882117034                   -0.53    8.0         
  2   -2.772143670694       -2.90       -1.30    1.0    157ms
  3   -2.772170272091       -4.58       -2.65    1.0    159ms
  4   -2.772170720507       -6.35       -3.96    2.0    207ms
  5   -2.772170722904       -8.62       -4.51    2.0    176ms
  6   -2.772170723013       -9.96       -5.51    1.0    198ms
  7   -2.772170723015      -11.67       -6.42    2.0    181ms
  8   -2.772170723015      -13.42       -6.86    2.0    197ms
  9   -2.772170723015   +  -15.05       -7.72    1.0    186ms
 10   -2.772170723015      -14.10       -8.81    2.0    196ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770837329642                   -0.52    9.0         
  2   -2.772063187912       -2.91       -1.32    1.0    185ms
  3   -2.7

1.7735580261890493

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770758958566                   -0.52    9.0         
  2   -2.772058045734       -2.89       -1.32    1.0    153ms
  3   -2.772083094913       -4.60       -2.49    1.0    201ms
  4   -2.772083345799       -6.60       -3.23    1.0    158ms
  5   -2.772083414273       -7.16       -3.75    2.0    184ms
  6   -2.772083417638       -8.47       -4.60    1.0    210ms
  7   -2.772083417807       -9.77       -5.20    2.0    180ms
  8   -2.772083417810      -11.50       -6.32    1.0    180ms
  9   -2.772083417811      -12.82       -6.77    2.0    197ms
 10   -2.772083417811   +  -14.57       -7.43    1.0    163ms
 11   -2.772083417811   +  -14.45       -8.26    1.0    196ms

Polarizability via ForwardDiff:       1.7725349832326425
Polarizability via finite difference: 1.7735580261890493
